In [1]:
import pandas as pd
import numpy as np
from functools import reduce
import statsmodels.api as sm

## Read in 2003 Data

In [2]:
#Read in files 2003 Consumption
consump_03_01 = pd.read_csv('2003_Consumption/FILE01.csv')
consump_03_01.columns = [col.strip() for col in consump_03_01.columns]
consump_03_01 = consump_03_01[['PUBID8', 'REGION8', 'CENDIV8', 'SQFT8', 'PBA8', 'YRCON8', 'FREESTN8'
                              ,'GLSSPC8', 'NELVTR8', 'NESLTR8', 'OPEN248', 'WKHRS8',
                              'NWKER8','OPNWE8']]

consump_03_02 = pd.read_csv('2003_Consumption/FILE02.csv')
consump_03_02.columns = [col.strip() for col in consump_03_02.columns]
consump_03_02 = consump_03_02[['PUBID8', 'ONEACT8']]

consump_03_04 = pd.read_csv('2003_Consumption/FILE04.csv')
consump_03_04.columns = [col.strip() for col in consump_03_04.columns]
consump_03_04 = consump_03_04[['PUBID8','RFGEQP8']]

consump_03_05 = pd.read_csv('2003_Consumption/FILE05.csv')
consump_03_05.columns = [col.strip() for col in consump_03_05.columns]
consump_03_05 = consump_03_05[['PUBID8','ELHT18', 'ELCOOL8','ELWATR8','ELCOOK8','ELMANU8']]

consump_03_15 = pd.read_csv('2003_Consumption/FILE15.csv')
consump_03_15.columns = [col.strip() for col in consump_03_15.columns]
consump_03_15 = consump_03_15[['PUBID8','ELCNS8']]

#Merge Dataframes
dfs = [consump_03_01, consump_03_02, consump_03_04, consump_03_05, consump_03_15]
consump_03 = reduce(lambda left,right: pd.merge(left,right,on='PUBID8'), dfs)

consump_03['YEAR'] = 2003

## Read in 2012 Data

In [3]:
consump_12 = pd.read_csv('2012_public_use_data_aug2016.csv')

consump_12 = consump_12[['PUBID', 'REGION', 'CENDIV', 'SQFT', 'PBA','YRCON','FREESTN', 'NELVTR','NESLTR', 'GLSSPC',
          'OPEN24', 'WKHRS', 'NWKER', 'OPNWE', 'ONEACT', 'RFGEQP', 'ELHT1', 'ELCOOL', 'ELWATR', 'ELCOOK', 'ELMANU',
          'ELCNS']]

consump_12['YEAR'] = 2012

There are climate fields PUBCLIM in 2012 and CLIMAT in 2003 that refer to a climate type a building is located.  These are based on the number of heating and cooling days.  We could potentially use this as a consumption feature by tying zipcodes to NOAA data which has the heating/cooling days for each of the stations.

GlassPercent Categories are different, might need to standardize these if we actually use them

## Merge Years & Data Standarization 

In [4]:
for i in consump_03.columns:
    consump_03[i] = pd.to_numeric(consump_03[i], errors = 'coerce')
    
consump_03.columns = list(consump_12.columns)

consump_all = pd.concat([consump_12,consump_03])

In [5]:
PBA_Dict = {
1:'Vacant',
2:'Office',
4:'Laboratory',
5:'Nonrefrigerated warehouse',
6:'Food sales',
7:'Public order and safety',
8:'Outpatient health care',
11:'Refrigerated warehouse',
12:'Religious worship',
13:'Public assembly',
14:'Education',
15:'Food service',
16:'Inpatient health care',
17:'Nursing',
18:'Lodging',
23:'Strip shopping mall',
24:'Enclosed mall',
25:'Retail other than mall',
26:'Service',
91: 'Other'}

In [6]:
pba_list = []
for i in consump_all['PBA']:
    pba_list.append(PBA_Dict[i])

consump_all['PBA_Detail'] = pd.Series(pba_list)

In [7]:
consump_all['NELVTR'] = consump_all['NELVTR'].fillna(value=0)
consump_all['NESLTR'] = consump_all['NESLTR'].fillna(value=0)

In [8]:
#Filter for freestanding building with a single primary activity
consump_filtered = consump_all.loc[(consump_all['FREESTN'] == 1) & (consump_all['ONEACT'] == 1)]
consump_filtered = consump_filtered.dropna(axis=0, how = 'any')

binary_fix = ['OPEN24','RFGEQP', 'ELHT1','ELCOOL','ELWATR', 'ELCOOK', 'ELMANU', 'OPNWE']
for column in binary_fix:
    consump_filtered[column] = consump_filtered[column].replace(to_replace = 2, value = 0)
    
max_val_fix = ['NELVTR', 'NESLTR']
for column in binary_fix:
    consump_filtered[column] = consump_filtered[column].replace(to_replace = 995, value = 51)

In [9]:
consump_all.to_csv('merged_consumption.csv')

# Regression Modeling 

In [61]:
consump_filtered.columns

Index([u'PUBID', u'REGION', u'CENDIV', u'SQFT', u'PBA', u'YRCON', u'FREESTN',
       u'NELVTR', u'NESLTR', u'GLSSPC', u'OPEN24', u'WKHRS', u'NWKER',
       u'OPNWE', u'ONEACT', u'RFGEQP', u'ELHT1', u'ELCOOL', u'ELWATR',
       u'ELCOOK', u'ELMANU', u'ELCNS', u'YEAR', u'PBA_Detail'],
      dtype='object')

In [62]:
X = consump_filtered[['SQFT', 'WKHRS', 'NWKER','OPEN24','NELVTR', 'NESLTR','RFGEQP', 'ELHT1',
                      'ELCOOL', 'ELWATR', 'ELCOOK', 'ELMANU', 'OPNWE']]
y = consump_filtered['ELCNS']
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
#predictions = model.predict(X)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  ELCNS   R-squared:                       0.632
Model:                            OLS   Adj. R-squared:                  0.631
Method:                 Least Squares   F-statistic:                     750.2
Date:                Mon, 02 Apr 2018   Prob (F-statistic):               0.00
Time:                        11:51:55   Log-Likelihood:                -95866.
No. Observations:                5687   AIC:                         1.918e+05
Df Residuals:                    5673   BIC:                         1.919e+05
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -5.71e+05   2.65e+05     -2.154      0.031   -1.09e+06   -5.14e+04
SQFT          15.8728      0.409     38.835      0.000      15.072      16.674
WKHRS      -6242.8584   3115.092     -2.004      0.045   -1.23e+04    -136.088
NWKER       3590.0365    180.012     19.943      0.000    3237.145    3942.928
OPEN24      2.502e+06   3.69e+05      6.784      0.000    1.78e+06    3.22e+06
NELVTR      1.671e+04   1633.212     10.233      0.000    1.35e+04    1.99e+04
NESLTR      3917.6349   1847.927      2.120      0.034     294.992    7540.277
RFGEQP     -1.435e+05   2.05e+05     -0.701      0.484   -5.45e+05    2.58e+05
ELHT1       9.123e+04   1.55e+05      0.589      0.556   -2.12e+05    3.95e+05
ELCOOL      2.621e+05   2.28e+05      1.147      0.251   -1.86e+05     7.1e+05
ELWATR     -8.689e+04   1.46e+05     -0.595      0.552   -3.73e+05    1.99e+05
ELCOOK      2.588e+05   1.58e+05      1.639      0.101   -5.07e+04    5.68e+05
ELMANU      2.402e+05   3.58e+05      0.671      0.502   -4.61e+05    9.42e+05
OPNWE       4.297e+05   1.67e+05      2.573      0.010    1.02e+05    7.57e+05
==============================================================================
Omnibus:                    11495.911   Durbin-Watson:                   1.992
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         60215669.417
Skew:                          16.304   Prob(JB):                         0.00
Kurtosis:                     506.047   Cond. No.                     1.71e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.71e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

# Business Specific Models

In [63]:
#Office Model

office_df = consump_filtered.loc[consump_filtered['PBA_Detail'] == 'Office']

X = office_df[['SQFT', 'WKHRS', 'NWKER','OPEN24','NELVTR', 'NESLTR','RFGEQP', 'ELHT1',
                      'ELCOOL', 'ELWATR', 'ELCOOK', 'ELMANU', 'OPNWE']]
y = office_df['ELCNS']
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
#predictions = model.predict(X)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  ELCNS   R-squared:                       0.699
Model:                            OLS   Adj. R-squared:                  0.695
Method:                 Least Squares   F-statistic:                     200.9
Date:                Mon, 02 Apr 2018   Prob (F-statistic):          3.72e-282
Time:                        11:51:55   Log-Likelihood:                -18726.
No. Observations:                1139   AIC:                         3.748e+04
Df Residuals:                    1125   BIC:                         3.755e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -2.852e+05   7.19e+05     -0.397      0.692    -1.7e+06    1.13e+06
SQFT          11.4248      0.750     15.225      0.000       9.952      12.897
WKHRS      -1.244e+04   8399.036     -1.481      0.139   -2.89e+04    4037.813
NWKER       1894.9051    283.450      6.685      0.000    1338.755    2451.055
OPEN24      2.347e+06   9.64e+05      2.434      0.015    4.55e+05    4.24e+06
NELVTR      1.687e+04   2238.490      7.537      0.000    1.25e+04    2.13e+04
NESLTR     -1362.4837   3441.325     -0.396      0.692   -8114.621    5389.653
RFGEQP      1.977e+05   2.87e+05      0.688      0.491   -3.66e+05    7.61e+05
ELHT1       3.894e+05   2.25e+05      1.734      0.083   -5.13e+04     8.3e+05
ELCOOL      6.093e+05   5.59e+05      1.090      0.276   -4.87e+05    1.71e+06
ELWATR     -2.713e+05   2.23e+05     -1.214      0.225    -7.1e+05    1.67e+05
ELCOOK      2.372e+05    2.7e+05      0.879      0.380   -2.92e+05    7.67e+05
ELMANU     -1.949e+05   6.48e+05     -0.301      0.764   -1.47e+06    1.08e+06
OPNWE       6.215e+05   2.36e+05      2.633      0.009    1.58e+05    1.08e+06
==============================================================================
Omnibus:                     2090.106   Durbin-Watson:                   1.974
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          3445554.569
Skew:                          12.621   Prob(JB):                         0.00
Kurtosis:                     271.262   Cond. No.                     3.41e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.41e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [64]:
#Food Service Model

food_serve_df = consump_filtered.loc[consump_filtered['PBA_Detail'] == 'Religious worship']

X = food_serve_df[['SQFT', 'WKHRS', 'NWKER','OPEN24','NELVTR', 'NESLTR','RFGEQP', 'ELHT1',
                      'ELCOOL', 'ELWATR', 'ELCOOK', 'ELMANU', 'OPNWE']]
y = food_serve_df['ELCNS']
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
#predictions = model.predict(X)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  ELCNS   R-squared:                       0.983
Model:                            OLS   Adj. R-squared:                  0.983
Method:                 Least Squares   F-statistic:                     1433.
Date:                Mon, 02 Apr 2018   Prob (F-statistic):          5.71e-272
Time:                        11:51:56   Log-Likelihood:                -4723.6
No. Observations:                 330   AIC:                             9475.
Df Residuals:                     316   BIC:                             9528.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -2.229e+05   2.32e+05     -0.960      0.338    -6.8e+05    2.34e+05
SQFT          18.5058      0.447     41.413      0.000      17.627      19.385
WKHRS       -399.3867    967.068     -0.413      0.680   -2302.093    1503.320
NWKER      -7605.1931    264.034    -28.804      0.000   -8124.680   -7085.706
OPEN24      -7.31e+04   1.67e+05     -0.438      0.662   -4.02e+05    2.56e+05
NELVTR     -3.532e+05    3.8e+04     -9.302      0.000   -4.28e+05   -2.79e+05
NESLTR      1.223e+06   4.74e+04     25.792      0.000    1.13e+06    1.32e+06
RFGEQP     -7.172e+04   7.15e+04     -1.002      0.317   -2.12e+05    6.91e+04
ELHT1       3.455e+04   5.41e+04      0.639      0.523   -7.18e+04    1.41e+05
ELCOOL     -5.137e+04   8.69e+04     -0.591      0.555   -2.22e+05     1.2e+05
ELWATR      1.031e+05   4.97e+04      2.076      0.039    5394.925    2.01e+05
ELCOOK      3.179e+04   4.97e+04      0.640      0.523    -6.6e+04     1.3e+05
ELMANU      1.471e+05   2.94e+05      0.499      0.618   -4.32e+05    7.26e+05
OPNWE       1.492e+05   2.09e+05      0.713      0.477   -2.63e+05    5.61e+05
==============================================================================
Omnibus:                      135.708   Durbin-Watson:                   1.973
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5823.457
Skew:                          -0.920   Prob(JB):                         0.00
Kurtosis:                      23.497   Cond. No.                     1.81e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.81e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [65]:
#Nonrefrigerated warehouse Model

ware_nofridge_df = consump_filtered.loc[consump_filtered['PBA_Detail'] == 'Nonrefrigerated warehouse']

X = ware_nofridge_df[['SQFT', 'WKHRS', 'NWKER','OPEN24','NELVTR', 'NESLTR','RFGEQP', 'ELHT1',
                      'ELCOOL', 'ELWATR', 'ELCOOK', 'ELMANU','OPNWE']]
y = ware_nofridge_df['ELCNS']
model = sm.OLS(y, X).fit()
predictions = model.predict(X)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  ELCNS   R-squared:                       0.785
Model:                            OLS   Adj. R-squared:                  0.780
Method:                 Least Squares   F-statistic:                     167.6
Date:                Mon, 02 Apr 2018   Prob (F-statistic):          2.39e-189
Time:                        11:51:56   Log-Likelihood:                -9743.1
No. Observations:                 610   AIC:                         1.951e+04
Df Residuals:                     597   BIC:                         1.957e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
SQFT           3.6773      0.470      7.826      0.000       2.754       4.600
WKHRS       2461.1421   3163.130      0.778      0.437   -3751.073    8673.357
NWKER       8693.4056    837.705     10.378      0.000    7048.198    1.03e+04
OPEN24      -403.5932    4.3e+05     -0.001      0.999   -8.44e+05    8.44e+05
NELVTR     -2.831e+05   1.59e+05     -1.779      0.076   -5.96e+05    2.94e+04
NESLTR      2.356e+05   9.49e+04      2.482      0.013    4.92e+04    4.22e+05
RFGEQP      6.578e+04   2.18e+05      0.302      0.763   -3.63e+05    4.94e+05
ELHT1       2.898e+05   2.02e+05      1.435      0.152   -1.07e+05    6.87e+05
ELCOOL     -2.481e+05   2.44e+05     -1.015      0.311   -7.28e+05    2.32e+05
ELWATR     -2.615e+05   1.93e+05     -1.354      0.176   -6.41e+05    1.18e+05
ELCOOK     -1.685e+05   4.15e+05     -0.406      0.685   -9.83e+05    6.46e+05
ELMANU      5.631e+05   2.64e+05      2.131      0.033    4.42e+04    1.08e+06
OPNWE      -1.628e+05   2.13e+05     -0.763      0.446   -5.82e+05    2.56e+05
==============================================================================
Omnibus:                      932.291   Durbin-Watson:                   2.066
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           455263.892
Skew:                           8.366   Prob(JB):                         0.00
Kurtosis:                     135.786   Cond. No.                     1.43e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.43e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [66]:
model.pvalues

SQFT      2.299197e-14
WKHRS     4.368350e-01
NWKER     2.616030e-23
OPEN24    9.992510e-01
NELVTR    7.573651e-02
NESLTR    1.332501e-02
RFGEQP    7.630833e-01
ELHT1     1.519137e-01
ELCOOL    3.105243e-01
ELWATR    1.763452e-01
ELCOOK    6.846517e-01
ELMANU    3.346907e-02
OPNWE     4.460395e-01
dtype: float64

## Production Models

In [67]:
final_dependent = ['SQFT', 'WKHRS', 'NWKER','OPEN24','NELVTR', 'NESLTR',
                   'RFGEQP', 'ELHT1','ELCOOL', 'ELWATR', 'ELCOOK', 'ELMANU', 'OPNWE']

In [68]:
ols_coefficients = []
ols_pvals = []
business_vals = []

for i in PBA_Dict.keys():
    temp_df = consump_filtered.loc[consump_filtered['PBA_Detail'] == PBA_Dict[i]]
    
    X = temp_df[final_dependent]
    y = temp_df['ELCNS']
    X = sm.add_constant(X)
    
    model = sm.OLS(y, X).fit()    
    
    ols_coefficients.append(model.params) 
    ols_pvals.append(model.pvalues)
    business_vals.append(PBA_Dict[i])


In [69]:
business_df = pd.DataFrame(business_vals)
business_df.columns = ['business_type']

ols_coef_df = pd.DataFrame(ols_coefficients)
ols_coef_df = pd.merge(ols_coef_df,business_df, left_index = True, right_index = True)

ols_pvals_df = pd.DataFrame(ols_pvals)
ols_pvals_df = pd.merge(ols_pvals_df,business_df, left_index = True, right_index = True)

#Create Multiplication Table
ols_keepers = ols_pvals_df.where(ols_pvals_df <= 0.10, 0)
ols_keepers = ols_keepers.where(ols_keepers <= 0,1)
##Make all sqft significant
ols_keepers['SQFT'] = ols_keepers['SQFT'].replace(to_replace = 0, value = 1)

#Usable Coefficients
ols_usable = ols_coef_df*ols_keepers
ols_usable = ols_usable.drop(['const', 'business_type'], axis=1)
ols_usable = pd.merge(ols_usable, business_df, left_index = True, right_index = True)

In [70]:
# 20180401: Added by Vincent to generate conefficients file to be consumed in the data lake
# Generate csv . files with most updated coefficients
ols_usable.to_csv('../../Data/ols_usable.csv')

In [71]:
def annual_consumption(ols_usable, biz_type, wkrs, sqft, wkhrs = 40,heat=0, cook=0, cool = 0, manu = 0, water = 0, elvr = 0, 
                       escl=0, opn24=0, rfrd=0, opnwe=0):
    
    return (ols_usable.loc[ols_usable['business_type'] == biz_type]['ELCOOK'] * cook +
            ols_usable.loc[ols_usable['business_type'] == biz_type]['ELCOOL'] * cool +
            ols_usable.loc[ols_usable['business_type'] == biz_type]['ELHT1'] * heat +
            ols_usable.loc[ols_usable['business_type'] == biz_type]['ELMANU'] * manu +
            ols_usable.loc[ols_usable['business_type'] == biz_type]['ELWATR'] * water +
            ols_usable.loc[ols_usable['business_type'] == biz_type]['NELVTR'] * elvr +
            ols_usable.loc[ols_usable['business_type'] == biz_type]['NESLTR'] * escl +
            ols_usable.loc[ols_usable['business_type'] == biz_type]['NWKER'] * wkrs +
            ols_usable.loc[ols_usable['business_type'] == biz_type]['OPEN24'] * opn24 +
            ols_usable.loc[ols_usable['business_type'] == biz_type]['RFGEQP'] * rfrd +
            ols_usable.loc[ols_usable['business_type'] == biz_type]['SQFT'] * sqft +
            ols_usable.loc[ols_usable['business_type'] == biz_type]['WKHRS'] * wkhrs +
            ols_usable.loc[ols_usable['business_type'] == biz_type]['OPNWE'] * opnwe).iloc[0]
    